In [1]:
import json
import os

with open('/Users/marie/Documents/github/config.json', 'r') as f:
    config = json.load(f)

    for key in config:
        os.environ[key] = config[key]

In [2]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase

In [3]:
# ! pip install clickhouse-sqlalchemy

In [8]:
uri = 'clickhouse+native://localhost/ecommerce'
db = SQLDatabase.from_uri(uri)
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [9]:
db.get_usable_table_names()

['sessions', 'users']

In [10]:
db.run('SELECT 1')

'[(1,)]'

In [11]:
db.run("SELECT COUNT(1) FROM ecommerce.users WHERE (country = 'United Kingdom') AND (is_active = 1)")

'[(111469,)]'

In [52]:
# toolkit.get_tools()

In [12]:
agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-0613"),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

In [14]:
import langchain
langchain.debug = False

In [15]:
agent_executor.run("How many active customers from the United Kingdom do we have? ")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


sessions, users
Invoking: `sql_db_schema` with `users`



CREATE TABLE users (
	user_id UInt64, 
	country String, 
	is_active UInt8, 
	age UInt64
) ENGINE = Log

/*
3 rows from users table:
user_id	country	is_active	age
1000001	United Kingdom	0	70
1000002	France	1	87
1000003	France	1	88
*/
Invoking: `sql_db_query` with `SELECT COUNT(*) FROM users WHERE country = 'United Kingdom' AND is_active = 1`


[(111469,)]We have 111,469 active customers from the United Kingdom.

> Finished chain.


'We have 111,469 active customers from the United Kingdom.'

In [16]:
langchain.debug = True

In [17]:
agent_executor.run("How many active customers from the United Kingdom do we have? ")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How many active customers from the United Kingdom do we have? "
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct clickhouse query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check

'We have 111,469 active customers from the United Kingdom.'

In [18]:
print("System: You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct clickhouse query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nIf the question does not seem related to the database, just return \"I don't know\" as the answer.\n\nHuman: How many active customers from the United Kingdom do we have? \nAI: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.")

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct clickhouse query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just retur